In [1]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal &> /dev/null
# Install rtree - Geopandas requirment
!apt install python3-rtree &> /dev/null
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git &> /dev/null
# Install descartes - Geopandas requirment
!pip install descartes &> /dev/null
!pip install plotly -U &> /dev/null

In [2]:
from pathlib import Path
import zipfile
import folium
import geopandas as gpd
import pandas as pd
import shapely

In [ ]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

# ダウンロード済みzipの解凍
- 値を含むデータ
- 位置データを含むデータ

In [ ]:
p = Path('/content/drive/MyDrive/work/pycon_mini_shizu/data/data/value')
zips = list(p.glob('*.zip'))
zips

[PosixPath('/content/drive/MyDrive/work/pycon_mini_shizu/data/data/value/tblT000848C22.zip'),
 PosixPath('/content/drive/MyDrive/work/pycon_mini_shizu/data/data/value/tblT000849C22.zip'),
 PosixPath('/content/drive/MyDrive/work/pycon_mini_shizu/data/data/value/tblT000850C22.zip'),
 PosixPath('/content/drive/MyDrive/work/pycon_mini_shizu/data/data/value/tblT000851C22.zip'),
 PosixPath('/content/drive/MyDrive/work/pycon_mini_shizu/data/data/value/tblT000852C22.zip'),
 PosixPath('/content/drive/MyDrive/work/pycon_mini_shizu/data/data/value/tblT000853C22.zip'),
 PosixPath('/content/drive/MyDrive/work/pycon_mini_shizu/data/data/value/tblT000865C22.zip'),
 PosixPath('/content/drive/MyDrive/work/pycon_mini_shizu/data/data/value/tblT000866C22.zip'),
 PosixPath('/content/drive/MyDrive/work/pycon_mini_shizu/data/data/value/tblT000875C22.zip')]

In [ ]:
data_target = '/content/drive/MyDrive/work/pycon_mini_shizu/data/data/extracted_value'
gis_target = '/content/drive/MyDrive/work/pycon_mini_shizu/data/data/extracted_gis'

In [ ]:
for zip in zips:
    with zipfile.ZipFile(zip) as z:
        z.extractall(data_target)

In [ ]:
gis_path = Path('/content/drive/MyDrive/work/pycon_mini_shizu/data/data/gis/A002005212015DDSWC22.zip')
with zipfile.ZipFile(gis_path) as zip:
    zip.extractall(gis_target)

In [ ]:
# データの確認

gis = Path(gis_target)
gis_data = list(gis.glob('h27*'))
data_path = Path(data_target)
data_list = list(data_path.glob('*.txt'))
print('---- GIS files ----')
for gis_file in gis_data:
    print(gis_file.name)
print('----- data files -----')
for data_file in data_list:
    print(data_file.name)

---- GIS files ----
h27ka22.prj
h27ka22.shp
h27ka22.dbf
h27ka22.shx
----- data files -----
tblT000848C22.txt
tblT000849C22.txt
tblT000850C22.txt
tblT000851C22.txt
tblT000852C22.txt
tblT000853C22.txt
tblT000865C22.txt
tblT000866C22.txt
tblT000875C22.txt


## 各データをマージする

1. データごとのマージ（txt同士）
2. 位置データをデータとマージ（txtとshp）

In [ ]:
d_list = list()

for data_path in data_list:
    df = pd.read_csv(data_path, encoding='cp932')
    d_list.append(df)

In [ ]:
for i in range(len(d_list)):
    print(d_list[i].head())
    print(d_list[i].info())

      KEY_CODE  HYOSYO CITYNAME  ... T000848002  T000848003  T000848004
0          NaN     NaN      NaN  ...         　男          　女        世帯総数
1      22101.0     1.0    静岡市葵区  ...     121761      131832      101673
2  221010010.0     2.0    静岡市葵区  ...         68         112         111
3  221010020.0     2.0    静岡市葵区  ...         39          50          40
4  221010030.0     2.0    静岡市葵区  ...         54          88          67

[5 rows x 11 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5369 entries, 0 to 5368
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   KEY_CODE    5368 non-null   float64
 1   HYOSYO      5368 non-null   float64
 2   CITYNAME    5368 non-null   object 
 3   NAME        5324 non-null   object 
 4   HTKSYORI    5368 non-null   float64
 5   HTKSAKI     0 non-null      float64
 6   GASSAN      0 non-null      float64
 7   T000848001  5369 non-null   object 
 8   T000848002  5369 non-nu

In [ ]:
data = gpd.read_file('/content/drive/MyDrive/work/pycon_mini_shizu/data/data/extracted_gis/h27ka22.shp')
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5153 entries, 0 to 5152
Data columns (total 36 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   KEY_CODE    5153 non-null   object  
 1   PREF        5153 non-null   object  
 2   CITY        5153 non-null   object  
 3   S_AREA      5153 non-null   object  
 4   PREF_NAME   5153 non-null   object  
 5   CITY_NAME   5153 non-null   object  
 6   S_NAME      5151 non-null   object  
 7   KIGO_E      605 non-null    object  
 8   HCODE       5153 non-null   int64   
 9   AREA        5153 non-null   float64 
 10  PERIMETER   5153 non-null   float64 
 11  H27KAxx_    5153 non-null   int64   
 12  H27KAxx_ID  5153 non-null   int64   
 13  KEN         5153 non-null   object  
 14  KEN_NAME    5153 non-null   object  
 15  SITYO_NAME  0 non-null      object  
 16  GST_NAME    5153 non-null   object  
 17  CSS_NAME    2072 non-null   object  
 18  KIHON1      5153 non-null   object  
 19

In [ ]:
data

,KEY_CODE,PREF,CITY,S_AREA,PREF_NAME,CITY_NAME,S_NAME,KIGO_E,HCODE,AREA,PERIMETER,H27KAxx_,H27KAxx_ID,KEN,KEN_NAME,SITYO_NAME,GST_NAME,CSS_NAME,KIHON1,DUMMY1,KIHON2,KEYCODE1,KEYCODE2,AREA_MAX_F,KIGO_D,N_KEN,N_CITY,KIGO_I,MOJI,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1,geometry
0,22,22,219,000000,静岡県,下田市,None,None,8101,48473.270,1317.373,1123,1122,22,静岡県,None,下田市,None,0000,-,00,219000000,None,None,None,None,None,None,None,0,0,0,138.94161,34.57509,0000-00,"POLYGON ((138.93971 34.57573, 138.93990 34.575..."
1,221010010,22,101,001000,静岡県,葵区,追手町,None,8101,175736.710,2701.273,3556,3555,22,静岡県,None,静岡市,葵区,0010,-,00,101001000,1010010,M,None,None,None,None,追手町,16,180,111,138.38022,34.97776,0010-00,"POLYGON ((138.38006 34.97625, 138.37999 34.976..."
2,221010020,22,101,002000,静岡県,葵区,城内町,None,8101,62198.313,1141.082,1273,1272,22,静岡県,None,静岡市,葵区,0020,-,00,101002000,1010020,M,None,None,None,None,城内町,6,89,40,138.38254,34.98212,0020-00,"POLYGON ((138.38196 34.98145, 138.38143 34.981..."
3,221010030,22,101,003000,静岡県,葵区,駿府町,None,8101,143355.593,2049.506,4189,4188,22,静岡県,None,静岡市,葵区,0030,-,00,101003000,1010030,M,None,None,None,None,駿府町,3,142,67,138.38643,34.97869,0030-00,"POLYGON ((138.38727 34.97837, 138.38724 34.978..."
4,221010040,22,101,004000,静岡県,葵区,駿府公園,None,8101,190501.969,1714.820,5102,5101,22,静岡県,None,静岡市,葵区,0040,-,00,101004000,1010040,M,None,None,None,None,駿府公園,1,0,0,138.38315,34.97931,0040-00,"POLYGON ((138.38404 34.98125, 138.38562 34.980..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5148,22461022093,22,461,022093,静岡県,森町,睦実（福田地）,None,8101,363078.812,2816.407,3228,3227,22,静岡県,None,周智郡,森町,0220,-,93,461022093,461022093,M,None,None,None,None,睦実（福田地）,2,238,87,137.92666,34.82578,0220-93,"POLYGON ((137.93144 34.82779, 137.93135 34.827..."
5149,22461022094,22,461,022094,静岡県,森町,睦実（戸綿）,None,8101,299962.927,3196.398,4698,4697,22,静岡県,None,周智郡,森町,0220,-,94,461022094,461022094,M,None,None,None,None,睦実（戸綿）,5,264,102,137.93165,34.83247,0220-94,"POLYGON ((137.92889 34.82930, 137.92879 34.829..."
5150,22461022095,22,461,022095,静岡県,森町,睦実（北戸綿）,None,8101,1778866.198,7980.654,3598,3597,22,静岡県,None,周智郡,森町,0220,-,95,461022095,461022095,M,None,None,None,None,睦実（北戸綿）,4,337,127,137.94251,34.83251,0220-95,"POLYGON ((137.94763 34.83033, 137.94767 34.830..."
5151,22461022096,22,461,022096,静岡県,森町,睦実（南戸綿）,None,8101,1056207.894,5563.228,3382,3381,22,静岡県,None,周智郡,森町,0220,-,96,461022096,461022096,M,None,None,None,None,睦実（南戸綿）,2,232,54,137.93593,34.82265,0220-96,"POLYGON ((137.94271 34.82688, 137.94290 34.826..."


## データの大体の形

- 0行目にもデータの属性名が含まれる
- カラムは6列目まで入力データの属性名、それ以降は属性コード
- 0行目は属性コードの日本語名

## 処理方針

- カラム名には属性コードよりも日本語名を持ちたい（あとあと分かりやすい）
- すべてのデータをひとまとめにしておきたい（今回の場合、DBでなくcsvにしておく）
- データをまとめるキーには KEY_CODE が使えそう

## 注意

- データと位置データで KEY_CODE のデータ型が違う！（あとから直す）


In [ ]:
def data_prepro(path: Path) -> pd.DataFrame:
    '''
    params
    path: Path
        読み込むデータのPath
    returns

    カラム名に日本語を付け加えて整形したデータフレーム
    '''
    df = pd.read_csv(path, encoding='cp932')
    first_col = list(df.columns[:7])
    second_col = list(df.iloc[0, 7:])
    col_name = first_col + second_col
    df.columns = col_name
    df = df.loc[1:, :]
    return df

In [ ]:
df1 = data_prepro(data_list[1])

In [ ]:
main_data = pd.DataFrame()
col_counts = 0 # 全部マージできたか確認用（カラム数カウント）

for i in range(len(data_list)):
    df = data_prepro(data_list[i])
    if i == 0:
        col_counts += df.shape[1]
        main_data = df.copy()
    else:
        df = df.drop(['HYOSYO', 'CITYNAME', 'NAME', 'HTKSYORI', 'HTKSAKI', 'GASSAN'], axis=1)
        col_counts += df.shape[1] - 1
        main_data = main_data.merge(df, on='KEY_CODE')

In [ ]:
main_data.shape

(5368, 144)

In [ ]:
col_counts

144

In [ ]:
main_data.head()

,KEY_CODE,HYOSYO,CITYNAME,NAME,HTKSYORI,HTKSAKI,GASSAN,人口総数,男,女,世帯総数,総数、年齢「不詳」含む,総数０～４歳,総数５～９歳,総数１０～１４歳,総数１５～１９歳,総数２０～２４歳,総数２５～２９歳,総数３０～３４歳,総数３５～３９歳,総数４０～４４歳,総数４５～４９歳,総数５０～５４歳,総数５５～５９歳,総数６０～６４歳,総数６５～６９歳,総数７０～７４歳,総数１５歳未満,総数１５～６４歳,総数６５歳以上,総数７５歳以上,男の総数、年齢「不詳」含む,男０～４歳,男５～９歳,男１０～１４歳,男１５～１９歳,男２０～２４歳,男２５～２９歳,男３０～３４歳,男３５～３９歳,男４０～４４歳,男４５～４９歳,男５０～５４歳,男５５～５９歳,男６０～６４歳,男６５～６９歳,男７０～７４歳,男１５歳未満,男１５～６４歳,男６５歳以上,男７５歳以上,女の総数、年齢「不詳」含む,女０～４歳,女５～９歳,女１０～１４歳,女１５～１９歳,女２０～２４歳,女２５～２９歳,女３０～３４歳,女３５～３９歳,女４０～４４歳,女４５～４９歳,女５０～５４歳,女５５～５９歳,女６０～６４歳,女６５～６９歳,女７０～７４歳,女１５歳未満,女１５～６４歳,女６５歳以上,女７５歳以上,一般世帯数（世帯人員６人以上含む）,世帯人員１人,世帯人員２人,世帯人員３人,世帯人員４人,世帯人員５人,一般世帯人員,１世帯当り人員,一般世帯総数,親族世帯,核家族世帯,うち夫婦のみの世帯,うち夫婦と子供から成る世帯,核家族以外の世帯,６歳未満世帯員のいる一般世帯総数,１８歳未満世帯員のいる一般世帯総数,６５歳以上世帯員のいる一般世帯総数,住宅に住む一般世帯,持ち家,民営借家,主世帯数,一戸建,長屋建,共同住宅,共同住宅１・２階建,共同住宅３～５階建,共同住宅６～１０階建,共同住宅１１階建以上,その他,総数_x,Ａ 農業、林業,うち農業,Ｂ 漁業,Ｃ 鉱業、採石業、砂利採取業,Ｄ 建設業,Ｅ 製造業,Ｆ 電気・ガス・熱供給・水道業,Ｇ 情報通信業,Ｈ 運輸業、郵便業,Ｉ 卸売業、小売業,Ｊ 金融業、保険業,Ｋ 不動産業、物品賃貸業,Ｌ 学術研究、専門・技術サービス業,Ｍ 宿泊業、飲食サービス業,Ｎ 生活関連サービス業、娯楽業,Ｏ 教育、学習支援業,Ｐ 医療、福祉,Ｑ 複合サービス事業,Ｒ サービス業（他に分類されないもの）,Ｓ 公務（他に分類されるものを除く）,Ｔ 分類不能の産業,雇用者（役員を含む）,自営業主（家庭内職者を含む）,家族従業者,総数_y,Ａ 管理的職業従事者,Ｂ 専門的・技術的職業従事者,Ｃ 事務従事者,Ｄ 販売従事者,Ｅ サービス職業従事者,Ｆ 保安職業従事者,Ｇ 農林漁業従事者,Ｈ 生産工程従事者,Ｉ 輸送・機械運転従事者,Ｊ 建設・採掘従事者,Ｋ 運搬・清掃・包装等従事者,Ｌ 分類不能の職業,総数,農林漁業就業者世帯,農林漁業・非農林漁業就業者混合世帯,非農林漁業就業者世帯,非就業者世帯,分類不能の世帯
0,22101.0,1.0,静岡市葵区,NaN,0.0,NaN,NaN,253593,121761,131832,101673,253593,9663,10456,10977,10829,9675,11604,13713,15455,19149,17482,16293,15683,17458,20167,17267,31096,147341,74152,36718,121761,4969,5432,5571,5543,4646,5715,6793,7855,9613,8722,8074,7697,8551,9576,7994,15972,73209,31911,14341,131832,4694,5024,5406,5286,5029,5889,6920,7600,9536,8760,8219,7986,8907,10591,9273,15124,74132,42241,22377,101488,30143,30153,19362,14151,5073,247170,2.4354603500,101488,70554,59477,21377,28018,11077,8952,22647,47629,100538,65457,26536,99707,61089,1096,37320,9986,19371,5766,2197,202,125663,3336,3139,23,36,10424,15997,721,2964,6123,23552,3999,2615,4196,6660,4430,6067,15966,824,8120,4125,5485,103751,12727,4832,125663,3384,19575,24404,18597,14868,2115,3234,14977,4161,6558,8480,5310,101488,968,1182,68782,26481,4075
1,221010010.0,2.0,静岡市葵区,追手町,0.0,NaN,NaN,180,68,112,111,180,2,5,3,3,3,3,7,13,10,4,12,9,18,27,22,10,82,88,39,68,-,1,3,2,-,1,2,9,5,2,3,1,8,13,6,4,33,31,12,112,2,4,-,1,3,2,5,4,5,2,9,8,10,14,16,6,49,57,27,107,64,29,10,3,1,169,1.5794392523,107,41,39,19,11,2,3,9,65,105,82,19,103,11,-,90,-,4,-,86,2,85,-,-,-,-,-,1,1,5,-,13,3,6,12,8,7,6,11,-,7,2,3,62,17,4,85,1,19,18,15,22,1,-,2,-,-,4,3,107,-,-,61,44,2
2,221010020.0,2.0,静岡市葵区,城内町,0.0,NaN,NaN,89,39,50,40,89,3,5,3,2,2,2,3,5,1,6,8,5,9,8,6,11,43,35,21,39,2,3,-,1,2,1,2,3,1,2,4,2,4,5,3,5,22,12,4,50,1,2,3,1,-,1,1,2,-,4,4,3,5,3,3,6,21,23,17,40,12,17,3,7,-,89,2.2250000000,40,28,25,12,8,3,3,7,28,40,38,2,40,40,-,-,-,-,-,-,-,41,-,-,-,-,4,3,-,1,-,9,6,6,-,2,2,3,3,-,1,1,-,36,5,-,41,1,7,9,12,3,-,-,2,1,3,3,-,40,-,-,24,16,-
3,221010030.0,2.0,静岡市葵区,駿府町,0.0,NaN,NaN,142,54,88,67,142,6,5,6,4,3,1,3,8,14,9,10,11,8,14,12,17,71,51,25,54,1,3,1,1,2,1,1,5,3,3,4,4,4,5,7,5,28,20,8,88,5,2,5,3,1,-,2,3,11,6,6,7,4,9,5,12,43,31,17,67,23,25,9,8,2,142,2.1194029851,67,43,34,14,10,9,4,12,36,66,51,14,66,45,1,19,-,19,-,-,1,79,-,-,-,-,3,2,-,1,1,29,1,2,6,14,5,3,2,-,3,1,6,38,25,12,79,-,6,8,26,21,-,-,6,-,2,4,6,67,-,-,40,23,4
4,221010040.0,2.0,静岡市葵区,駿府公園,0.0,NaN,NaN,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


In [ ]:
main_data.sort_values('KEY_CODE')

,KEY_CODE,HYOSYO,CITYNAME,NAME,HTKSYORI,HTKSAKI,GASSAN,人口総数,男,女,世帯総数,総数、年齢「不詳」含む,総数０～４歳,総数５～９歳,総数１０～１４歳,総数１５～１９歳,総数２０～２４歳,総数２５～２９歳,総数３０～３４歳,総数３５～３９歳,総数４０～４４歳,総数４５～４９歳,総数５０～５４歳,総数５５～５９歳,総数６０～６４歳,総数６５～６９歳,総数７０～７４歳,総数１５歳未満,総数１５～６４歳,総数６５歳以上,総数７５歳以上,男の総数、年齢「不詳」含む,男０～４歳,男５～９歳,男１０～１４歳,男１５～１９歳,男２０～２４歳,男２５～２９歳,男３０～３４歳,男３５～３９歳,男４０～４４歳,男４５～４９歳,男５０～５４歳,男５５～５９歳,男６０～６４歳,男６５～６９歳,男７０～７４歳,男１５歳未満,男１５～６４歳,男６５歳以上,男７５歳以上,女の総数、年齢「不詳」含む,女０～４歳,女５～９歳,女１０～１４歳,女１５～１９歳,女２０～２４歳,女２５～２９歳,女３０～３４歳,女３５～３９歳,女４０～４４歳,女４５～４９歳,女５０～５４歳,女５５～５９歳,女６０～６４歳,女６５～６９歳,女７０～７４歳,女１５歳未満,女１５～６４歳,女６５歳以上,女７５歳以上,一般世帯数（世帯人員６人以上含む）,世帯人員１人,世帯人員２人,世帯人員３人,世帯人員４人,世帯人員５人,一般世帯人員,１世帯当り人員,一般世帯総数,親族世帯,核家族世帯,うち夫婦のみの世帯,うち夫婦と子供から成る世帯,核家族以外の世帯,６歳未満世帯員のいる一般世帯総数,１８歳未満世帯員のいる一般世帯総数,６５歳以上世帯員のいる一般世帯総数,住宅に住む一般世帯,持ち家,民営借家,主世帯数,一戸建,長屋建,共同住宅,共同住宅１・２階建,共同住宅３～５階建,共同住宅６～１０階建,共同住宅１１階建以上,その他,総数_x,Ａ 農業、林業,うち農業,Ｂ 漁業,Ｃ 鉱業、採石業、砂利採取業,Ｄ 建設業,Ｅ 製造業,Ｆ 電気・ガス・熱供給・水道業,Ｇ 情報通信業,Ｈ 運輸業、郵便業,Ｉ 卸売業、小売業,Ｊ 金融業、保険業,Ｋ 不動産業、物品賃貸業,Ｌ 学術研究、専門・技術サービス業,Ｍ 宿泊業、飲食サービス業,Ｎ 生活関連サービス業、娯楽業,Ｏ 教育、学習支援業,Ｐ 医療、福祉,Ｑ 複合サービス事業,Ｒ サービス業（他に分類されないもの）,Ｓ 公務（他に分類されるものを除く）,Ｔ 分類不能の産業,雇用者（役員を含む）,自営業主（家庭内職者を含む）,家族従業者,総数_y,Ａ 管理的職業従事者,Ｂ 専門的・技術的職業従事者,Ｃ 事務従事者,Ｄ 販売従事者,Ｅ サービス職業従事者,Ｆ 保安職業従事者,Ｇ 農林漁業従事者,Ｈ 生産工程従事者,Ｉ 輸送・機械運転従事者,Ｊ 建設・採掘従事者,Ｋ 運搬・清掃・包装等従事者,Ｌ 分類不能の職業,総数,農林漁業就業者世帯,農林漁業・非農林漁業就業者混合世帯,非農林漁業就業者世帯,非就業者世帯,分類不能の世帯
0,2.210100e+04,1.0,静岡市葵区,NaN,0.0,NaN,NaN,253593,121761,131832,101673,253593,9663,10456,10977,10829,9675,11604,13713,15455,19149,17482,16293,15683,17458,20167,17267,31096,147341,74152,36718,121761,4969,5432,5571,5543,4646,5715,6793,7855,9613,8722,8074,7697,8551,9576,7994,15972,73209,31911,14341,131832,4694,5024,5406,5286,5029,5889,6920,7600,9536,8760,8219,7986,8907,10591,9273,15124,74132,42241,22377,101488,30143,30153,19362,14151,5073,247170,2.4354603500,101488,70554,59477,21377,28018,11077,8952,22647,47629,100538,65457,26536,99707,61089,1096,37320,9986,19371,5766,2197,202,125663,3336,3139,23,36,10424,15997,721,2964,6123,23552,3999,2615,4196,6660,4430,6067,15966,824,8120,4125,5485,103751,12727,4832,125663,3384,19575,24404,18597,14868,2115,3234,14977,4161,6558,8480,5310,101488,968,1182,68782,26481,4075
457,2.210200e+04,1.0,静岡市駿河区,NaN,0.0,NaN,NaN,212419,105257,107162,90631,212419,8558,8798,9153,10249,11069,11315,12654,14107,17001,15505,13455,12079,13130,15103,12713,26509,130564,53531,25715,105257,4409,4530,4732,5315,5701,5925,6546,7242,8675,7953,6833,6142,6469,7272,5904,13671,66801,23473,10297,107162,4149,4268,4421,4934,5368,5390,6108,6865,8326,7552,6622,5937,6661,7831,6809,12838,63763,30058,15418,90537,32425,24346,16142,11761,3889,208618,2.3042292102,90537,57126,49045,17380,23837,8081,8062,19506,34828,89277,49121,33272,88649,45446,1128,41921,15037,19213,5093,2578,154,107551,1770,1731,112,13,8788,16854,577,2952,5360,19697,3106,2277,3197,6585,3816,5100,10827,777,6448,3663,5632,90902,8949,3100,107551,2641,15400,21243,16896,12728,1998,1851,13811,3280,5076,7194,5433,90537,538,680,61171,23852,4296
664,2.210300e+04,1.0,静岡市清水区,NaN,0.0,NaN,NaN,238977,116320,122657,93709,238977,8217,9083,10394,11232,9896,10569,11756,13982,17502,16358,15158,14248,16860,20416,17368,27694,137561,72546,34762,116320,4216,4606,5333,5950,5114,5467,6042,7067,8931,8244,7658,7054,8354,9953,8234,14155,69881,31408,13221,122657,4001,4477,5061,5282,4782,5102,5714,6915,8571,8114,7500,7194,8506,10463,9134,13539,67680,41138,21541,93601,26493,27143,18328,13338,5188,234883,2.5094069508,93601,66364,53523,18865,25102,12841,7600,20558,47471,92318,64778,22269,91553,63948,1342,26125,12618,10394,2065,1048,138,117638,3441,3389,372,6,10215,26055,606,1844,8275,18808,2696,1662,2960,5704,3841,4781,12114,732,7160,2430,3936,98763,11234,4594,117638,2664,14863,21812,14511,13052,1623,3686,22011,4678,5953,9053,3732,93601,1043,1384,62860,25461,2853
996,2.213100e+04,1.0,浜松市中区,NaN,0.0,NaN,NaN,237443,118259,119184,106191,237443,9585,9741,10208,11032,11515,12694,13794,15093,18436,16889,15688,14583,14829,16518,13326,29534,144553,57891,28047,118259,4896,4874,5342,5779,6339,6719,7018,7645,9372,8655,7994,7479,7413,8009,6187,15112,74413

## 問題

- 位置データのKEY_CODEかぶっている問題
- IDは唯一ではないのだった・・・

In [ ]:
main_data['KEY_CODE'].value_counts()

22213075002    1
221031100      1
221340230      1
22101048002    1
22210147011    1
              ..
22221041015    1
22344007029    1
222066040      1
221011321      1
221020160      1
Name: KEY_CODE, Length: 5368, dtype: int64

In [ ]:
data['KEY_CODE'].value_counts()

222190630      12
222101090      11
222030340       9
222030330       9
22210095001     8
               ..
22342090017     1
22131028002     1
223050060       1
22220027001     1
221020160       1
Name: KEY_CODE, Length: 4778, dtype: int64

In [ ]:
# KEY_CODEが重なっている数
key_counts = data['KEY_CODE'].value_counts().reset_index()
len(key_counts.query('KEY_CODE > 1'))

230

In [ ]:
most = key_counts.loc[2, 'index']
most_data = data.query(f'KEY_CODE == {most}')
most_data.shape

(9, 36)

In [ ]:
most_data

,KEY_CODE,PREF,CITY,S_AREA,PREF_NAME,CITY_NAME,S_NAME,KIGO_E,HCODE,AREA,PERIMETER,H27KAxx_,H27KAxx_ID,KEN,KEN_NAME,SITYO_NAME,GST_NAME,CSS_NAME,KIHON1,DUMMY1,KIHON2,KEYCODE1,KEYCODE2,AREA_MAX_F,KIGO_D,N_KEN,N_CITY,KIGO_I,MOJI,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1,geometry
1834,222030340,22,203,034000,静岡県,沼津市,西熊堂,E1,8101,106487.222,1974.791,1076,1075,22,静岡県,None,沼津市,None,0340,-,00,203034000,2030340,None,None,None,None,None,西熊堂,25,1110,441,138.85946,35.12417,0340-00,"POLYGON ((138.85972 35.12183, 138.85982 35.121..."
1835,222030340,22,203,034000,静岡県,沼津市,西熊堂,E2,8101,328145.771,5063.686,1075,1074,22,静岡県,None,沼津市,None,0340,-,00,203034000,2030340,M,None,None,None,None,西熊堂,0,0,0,138.85696,35.13796,0340-00,"POLYGON ((138.85718 35.13429, 138.85708 35.134..."
1836,222030340,22,203,034000,静岡県,沼津市,西熊堂,E3,8101,9713.674,741.265,1074,1073,22,静岡県,None,沼津市,None,0340,-,00,203034000,2030340,None,None,None,None,None,西熊堂,0,0,0,138.85982,35.12963,0340-00,"POLYGON ((138.85940 35.13100, 138.85957 35.130..."
1837,222030340,22,203,034000,静岡県,沼津市,西熊堂,E4,8101,5148.242,408.728,1073,1072,22,静岡県,None,沼津市,None,0340,-,00,203034000,2030340,None,None,None,None,None,西熊堂,0,0,0,138.86288,35.13155,0340-00,"POLYGON ((138.86343 35.13120, 138.86332 35.131..."
1838,222030340,22,203,034000,静岡県,沼津市,西熊堂,E5,8101,6256.315,526.908,1071,1070,22,静岡県,None,沼津市,None,0340,-,00,203034000,2030340,None,None,None,None,None,西熊堂,0,0,0,138.86149,35.12688,0340-00,"POLYGON ((138.86167 35.12772, 138.86176 35.127..."
1839,222030340,22,203,034000,静岡県,沼津市,西熊堂,E6,8101,7101.050,329.680,1070,1069,22,静岡県,None,沼津市,None,0340,-,00,203034000,2030340,None,None,None,None,None,西熊堂,0,0,0,138.86442,35.12277,0340-00,"POLYGON ((138.86481 35.12289, 138.86487 35.122..."
1840,222030340,22,203,034000,静岡県,沼津市,西熊堂,E7,8101,1296.075,144.617,1069,1068,22,静岡県,None,沼津市,None,0340,-,00,203034000,2030340,None,None,None,None,None,西熊堂,0,0,0,138.86437,35.12407,0340-00,"POLYGON ((138.86449 35.12427, 138.86454 35.124..."
1841,222030340,22,203,034000,静岡県,沼津市,西熊堂,E8,8101,394.779,81.249,1072,1071,22,静岡県,None,沼津市,None,0340,-,00,203034000,2030340,None,None,None,None,None,西熊堂,0,0,0,138.86147,35.12503,0340-00,"POLYGON ((138.86132 35.12496, 138.86134 35.125..."
1842,222030340,22,203,034000,静岡県,沼津市,西熊堂,E9,8101,1604.682,163.468,1068,1067,22,静岡県,None,沼津市,None,0340,-,00,203034000,2030340,None,None,None,None,None,西熊堂,0,0,0,138.86742,35.12124,0340-00,"POLYGON ((138.86726 35.12111, 138.86714 35.121..."


In [ ]:
most_data1 = most_data[['KEY_CODE', 'JINKO', 'KIGO_E', 'geometry']]
most_data1 = most_data.to_crs('EPSG:4326')
center = shapely.geometry.MultiPolygon(most_data1.geometry.values).centroid
m = folium.Map([center.y, center.x], 
                zoom_start=14,
                #tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
                #attr='google map'
               )

folium.Choropleth(
geo_data=most_data1,
data=most_data1,
columns=['KEY_CODE', 'JINKO'],
key_on=f'feature.properties.KEY_CODE',
fill_color='OrRd',
name='labels'
).add_to(m)

folium.GeoJson(
    most_data1, 
    tooltip=folium.features.GeoJsonTooltip(fields=['KEY_CODE', "KIGO_E","JINKO"])
).add_to(m)

folium.LayerControl().add_to(m)

m

## KEY_CODE複数の場所

- 飛び地になっている
- それをKIGO_Eを使って複数表現している（E1 - En）
- 人口、世帯数はE!に設定してある

In [ ]:
# geopandasのdissolveを使ってKEY_CODE1つにまとめる

data_dissoleved = data.dissolve(by='KEY_CODE', aggfunc='first', as_index=False)
data_dissoleved

,KEY_CODE,geometry,PREF,CITY,S_AREA,PREF_NAME,CITY_NAME,S_NAME,KIGO_E,HCODE,AREA,PERIMETER,H27KAxx_,H27KAxx_ID,KEN,KEN_NAME,SITYO_NAME,GST_NAME,CSS_NAME,KIHON1,DUMMY1,KIHON2,KEYCODE1,KEYCODE2,AREA_MAX_F,KIGO_D,N_KEN,N_CITY,KIGO_I,MOJI,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1
0,22,"POLYGON ((138.93971 34.57573, 138.93990 34.575...",22,219,000000,静岡県,下田市,None,None,8101,48473.270,1317.373,1123,1122,22,静岡県,None,下田市,None,0000,-,00,219000000,None,None,None,None,None,None,None,0,0,0,138.94161,34.57509,0000-00
1,221010010,"POLYGON ((138.38006 34.97625, 138.37999 34.976...",22,101,001000,静岡県,葵区,追手町,None,8101,175736.710,2701.273,3556,3555,22,静岡県,None,静岡市,葵区,0010,-,00,101001000,1010010,M,None,None,None,None,追手町,16,180,111,138.38022,34.97776,0010-00
2,221010020,"POLYGON ((138.38196 34.98145, 138.38143 34.981...",22,101,002000,静岡県,葵区,城内町,None,8101,62198.313,1141.082,1273,1272,22,静岡県,None,静岡市,葵区,0020,-,00,101002000,1010020,M,None,None,None,None,城内町,6,89,40,138.38254,34.98212,0020-00
3,221010030,"POLYGON ((138.38727 34.97837, 138.38724 34.978...",22,101,003000,静岡県,葵区,駿府町,None,8101,143355.593,2049.506,4189,4188,22,静岡県,None,静岡市,葵区,0030,-,00,101003000,1010030,M,None,None,None,None,駿府町,3,142,67,138.38643,34.97869,0030-00
4,221010040,"POLYGON ((138.38404 34.98125, 138.38562 34.980...",22,101,004000,静岡県,葵区,駿府公園,None,8101,190501.969,1714.820,5102,5101,22,静岡県,None,静岡市,葵区,0040,-,00,101004000,1010040,M,None,None,None,None,駿府公園,1,0,0,138.38315,34.97931,0040-00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4773,22461022093,"POLYGON ((137.93144 34.82779, 137.93135 34.827...",22,461,022093,静岡県,森町,睦実（福田地）,None,8101,363078.812,2816.407,3228,3227,22,静岡県,None,周智郡,森町,0220,-,93,461022093,461022093,M,None,None,None,None,睦実（福田地）,2,238,87,137.92666,34.82578,0220-93
4774,22461022094,"POLYGON ((137.92889 34.82930, 137.92879 34.829...",22,461,022094,静岡県,森町,睦実（戸綿）,None,8101,299962.927,3196.398,4698,4697,22,静岡県,None,周智郡,森町,0220,-,94,461022094,461022094,M,None,None,None,None,睦実（戸綿）,5,264,102,137.93165,34.83247,0220-94
4775,22461022095,"POLYGON ((137.94763 34.83033, 137.94767 34.830...",22,461,022095,静岡県,森町,睦実（北戸綿）,None,8101,1778866.198,7980.654,3598,3597,22,静岡県,None,周智郡,森町,0220,-,95,461022095,461022095,M,None,None,None,None,睦実（北戸綿）,4,337,127,137.94251,34.83251,0220-95
4776,22461022096,"POLYGON ((137.94271 34.82688, 137.94290 34.826...",22,461,022096,静岡県,森町,睦実（南戸綿）,None,8101,1056207.894,5563.228,3382,3381,22,静岡県,None,周智郡,森町,0220,-,96,461022096,461022096,M,None,None,None,None,睦実（南戸綿）,2,232,54,137.93593,34.82265,0220-96


## ↑のデータに位置データをくっつける

- KEY_CODEを利用する
- 位置データから入らないデータは落としておく（KEY_CODE, AREA(面積), JINKO(人口), geometry　に絞る）

In [ ]:
# データ型を合わせる
main_data['KEY_CODE'] = main_data['KEY_CODE'].astype(int)
data_dissoleved['KEY_CODE'] = data_dissoleved['KEY_CODE'].astype(int)

In [ ]:
data1 = data_dissoleved[['KEY_CODE', 'AREA', 'JINKO', 'geometry']]

In [ ]:
data2 = data1.merge(main_data, on='KEY_CODE')

In [ ]:
data2.shape

(4777, 147)

In [ ]:
data2.head()

,KEY_CODE,AREA,JINKO,geometry,HYOSYO,CITYNAME,NAME,HTKSYORI,HTKSAKI,GASSAN,人口総数,男,女,世帯総数,総数、年齢「不詳」含む,総数０～４歳,総数５～９歳,総数１０～１４歳,総数１５～１９歳,総数２０～２４歳,総数２５～２９歳,総数３０～３４歳,総数３５～３９歳,総数４０～４４歳,総数４５～４９歳,総数５０～５４歳,総数５５～５９歳,総数６０～６４歳,総数６５～６９歳,総数７０～７４歳,総数１５歳未満,総数１５～６４歳,総数６５歳以上,総数７５歳以上,男の総数、年齢「不詳」含む,男０～４歳,男５～９歳,男１０～１４歳,男１５～１９歳,男２０～２４歳,男２５～２９歳,男３０～３４歳,男３５～３９歳,男４０～４４歳,男４５～４９歳,男５０～５４歳,男５５～５９歳,男６０～６４歳,男６５～６９歳,男７０～７４歳,男１５歳未満,男１５～６４歳,男６５歳以上,男７５歳以上,女の総数、年齢「不詳」含む,女０～４歳,女５～９歳,女１０～１４歳,女１５～１９歳,女２０～２４歳,女２５～２９歳,女３０～３４歳,女３５～３９歳,女４０～４４歳,女４５～４９歳,女５０～５４歳,女５５～５９歳,女６０～６４歳,女６５～６９歳,女７０～７４歳,女１５歳未満,女１５～６４歳,女６５歳以上,女７５歳以上,一般世帯数（世帯人員６人以上含む）,世帯人員１人,世帯人員２人,世帯人員３人,世帯人員４人,世帯人員５人,一般世帯人員,１世帯当り人員,一般世帯総数,親族世帯,核家族世帯,うち夫婦のみの世帯,うち夫婦と子供から成る世帯,核家族以外の世帯,６歳未満世帯員のいる一般世帯総数,１８歳未満世帯員のいる一般世帯総数,６５歳以上世帯員のいる一般世帯総数,住宅に住む一般世帯,持ち家,民営借家,主世帯数,一戸建,長屋建,共同住宅,共同住宅１・２階建,共同住宅３～５階建,共同住宅６～１０階建,共同住宅１１階建以上,その他,総数_x,Ａ 農業、林業,うち農業,Ｂ 漁業,Ｃ 鉱業、採石業、砂利採取業,Ｄ 建設業,Ｅ 製造業,Ｆ 電気・ガス・熱供給・水道業,Ｇ 情報通信業,Ｈ 運輸業、郵便業,Ｉ 卸売業、小売業,Ｊ 金融業、保険業,Ｋ 不動産業、物品賃貸業,Ｌ 学術研究、専門・技術サービス業,Ｍ 宿泊業、飲食サービス業,Ｎ 生活関連サービス業、娯楽業,Ｏ 教育、学習支援業,Ｐ 医療、福祉,Ｑ 複合サービス事業,Ｒ サービス業（他に分類されないもの）,Ｓ 公務（他に分類されるものを除く）,Ｔ 分類不能の産業,雇用者（役員を含む）,自営業主（家庭内職者を含む）,家族従業者,総数_y,Ａ 管理的職業従事者,Ｂ 専門的・技術的職業従事者,Ｃ 事務従事者,Ｄ 販売従事者,Ｅ サービス職業従事者,Ｆ 保安職業従事者,Ｇ 農林漁業従事者,Ｈ 生産工程従事者,Ｉ 輸送・機械運転従事者,Ｊ 建設・採掘従事者,Ｋ 運搬・清掃・包装等従事者,Ｌ 分類不能の職業,総数,農林漁業就業者世帯,農林漁業・非農林漁業就業者混合世帯,非農林漁業就業者世帯,非就業者世帯,分類不能の世帯
0,221010010,175736.710,180,"POLYGON ((138.38006 34.97625, 138.37999 34.976...",2.0,静岡市葵区,追手町,0.0,NaN,NaN,180,68,112,111,180,2,5,3,3,3,3,7,13,10,4,12,9,18,27,22,10,82,88,39,68,-,1,3,2,-,1,2,9,5,2,3,1,8,13,6,4,33,31,12,112,2,4,-,1,3,2,5,4,5,2,9,8,10,14,16,6,49,57,27,107,64,29,10,3,1,169,1.5794392523,107,41,39,19,11,2,3,9,65,105,82,19,103,11,-,90,-,4,-,86,2,85,-,-,-,-,-,1,1,5,-,13,3,6,12,8,7,6,11,-,7,2,3,62,17,4,85,1,19,18,15,22,1,-,2,-,-,4,3,107,-,-,61,44,2
1,221010020,62198.313,89,"POLYGON ((138.38196 34.98145, 138.38143 34.981...",2.0,静岡市葵区,城内町,0.0,NaN,NaN,89,39,50,40,89,3,5,3,2,2,2,3,5,1,6,8,5,9,8,6,11,43,35,21,39,2,3,-,1,2,1,2,3,1,2,4,2,4,5,3,5,22,12,4,50,1,2,3,1,-,1,1,2,-,4,4,3,5,3,3,6,21,23,17,40,12,17,3,7,-,89,2.2250000000,40,28,25,12,8,3,3,7,28,40,38,2,40,40,-,-,-,-,-,-,-,41,-,-,-,-,4,3,-,1,-,9,6,6,-,2,2,3,3,-,1,1,-,36,5,-,41,1,7,9,12,3,-,-,2,1,3,3,-,40,-,-,24,16,-
2,221010030,143355.593,142,"POLYGON ((138.38727 34.97837, 138.38724 34.978...",2.0,静岡市葵区,駿府町,0.0,NaN,NaN,142,54,88,67,142,6,5,6,4,3,1,3,8,14,9,10,11,8,14,12,17,71,51,25,54,1,3,1,1,2,1,1,5,3,3,4,4,4,5,7,5,28,20,8,88,5,2,5,3,1,-,2,3,11,6,6,7,4,9,5,12,43,31,17,67,23,25,9,8,2,142,2.1194029851,67,43,34,14,10,9,4,12,36,66,51,14,66,45,1,19,-,19,-,-,1,79,-,-,-,-,3,2,-,1,1,29,1,2,6,14,5,3,2,-,3,1,6,38,25,12,79,-,6,8,26,21,-,-,6,-,2,4,6,67,-,-,40,23,4
3,221010040,190501.969,0,"POLYGON ((138.38404 34.98125, 138.38562 34.980...",2.0,静岡市葵区,駿府公園,0.0,NaN,NaN,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
4,221010050,165730.625,1209,"POLYGON ((138.37944 34.97951, 138.37932 34.979...",2.0,静岡市葵区,西草深町,0.0,NaN,NaN,1209,573,636,523,1209,34,48,41,51,36,43,56,64,97,78,74,93,91,107,101,123,683,394,186,573,17,27,24,26,19,23,28,30,43,40,35,46,40,52,48,68,330,168,68,636,17,21,17,25,17,20,28,34,54,38,39,47,51,55,53,55,353,226,118,523,166,164,98,65,21,1209,2.3116634799,523,352,302,127,130,50,34,95,271,517,332,150,508,343,2,162,14,133,15,-,1,593,4,2,-,-,29,59,4,24,13,109,21,30,39,37,21,25,77,4,35,27,35,443,84,37,593,39,142,126,96,59,4,4,40,8,18,22,35,523,1,2,343,155,22


## HYOSYO
- 地域区分番号
- 1: 市町村、2: 大字・町名、3: 字・丁目

In [ ]:
main_data['HYOSYO'].value_counts()

2.0    2992
3.0    2333
1.0      43
Name: HYOSYO, dtype: int64

In [ ]:
data2['HYOSYO'].value_counts()

2.0    2444
3.0    2333
Name: HYOSYO, dtype: int64

In [ ]:
data2.query('HYOSYO == 2')

,KEY_CODE,AREA,JINKO,geometry,HYOSYO,CITYNAME,NAME,HTKSYORI,HTKSAKI,GASSAN,人口総数,男,女,世帯総数,総数、年齢「不詳」含む,総数０～４歳,総数５～９歳,総数１０～１４歳,総数１５～１９歳,総数２０～２４歳,総数２５～２９歳,総数３０～３４歳,総数３５～３９歳,総数４０～４４歳,総数４５～４９歳,総数５０～５４歳,総数５５～５９歳,総数６０～６４歳,総数６５～６９歳,総数７０～７４歳,総数１５歳未満,総数１５～６４歳,総数６５歳以上,総数７５歳以上,男の総数、年齢「不詳」含む,男０～４歳,男５～９歳,男１０～１４歳,男１５～１９歳,男２０～２４歳,男２５～２９歳,男３０～３４歳,男３５～３９歳,男４０～４４歳,男４５～４９歳,男５０～５４歳,男５５～５９歳,男６０～６４歳,男６５～６９歳,男７０～７４歳,男１５歳未満,男１５～６４歳,男６５歳以上,男７５歳以上,女の総数、年齢「不詳」含む,女０～４歳,女５～９歳,女１０～１４歳,女１５～１９歳,女２０～２４歳,女２５～２９歳,女３０～３４歳,女３５～３９歳,女４０～４４歳,女４５～４９歳,女５０～５４歳,女５５～５９歳,女６０～６４歳,女６５～６９歳,女７０～７４歳,女１５歳未満,女１５～６４歳,女６５歳以上,女７５歳以上,一般世帯数（世帯人員６人以上含む）,世帯人員１人,世帯人員２人,世帯人員３人,世帯人員４人,世帯人員５人,一般世帯人員,１世帯当り人員,一般世帯総数,親族世帯,核家族世帯,うち夫婦のみの世帯,うち夫婦と子供から成る世帯,核家族以外の世帯,６歳未満世帯員のいる一般世帯総数,１８歳未満世帯員のいる一般世帯総数,６５歳以上世帯員のいる一般世帯総数,住宅に住む一般世帯,持ち家,民営借家,主世帯数,一戸建,長屋建,共同住宅,共同住宅１・２階建,共同住宅３～５階建,共同住宅６～１０階建,共同住宅１１階建以上,その他,総数_x,Ａ 農業、林業,うち農業,Ｂ 漁業,Ｃ 鉱業、採石業、砂利採取業,Ｄ 建設業,Ｅ 製造業,Ｆ 電気・ガス・熱供給・水道業,Ｇ 情報通信業,Ｈ 運輸業、郵便業,Ｉ 卸売業、小売業,Ｊ 金融業、保険業,Ｋ 不動産業、物品賃貸業,Ｌ 学術研究、専門・技術サービス業,Ｍ 宿泊業、飲食サービス業,Ｎ 生活関連サービス業、娯楽業,Ｏ 教育、学習支援業,Ｐ 医療、福祉,Ｑ 複合サービス事業,Ｒ サービス業（他に分類されないもの）,Ｓ 公務（他に分類されるものを除く）,Ｔ 分類不能の産業,雇用者（役員を含む）,自営業主（家庭内職者を含む）,家族従業者,総数_y,Ａ 管理的職業従事者,Ｂ 専門的・技術的職業従事者,Ｃ 事務従事者,Ｄ 販売従事者,Ｅ サービス職業従事者,Ｆ 保安職業従事者,Ｇ 農林漁業従事者,Ｈ 生産工程従事者,Ｉ 輸送・機械運転従事者,Ｊ 建設・採掘従事者,Ｋ 運搬・清掃・包装等従事者,Ｌ 分類不能の職業,総数,農林漁業就業者世帯,農林漁業・非農林漁業就業者混合世帯,非農林漁業就業者世帯,非就業者世帯,分類不能の世帯
0,221010010,175736.710,180,"POLYGON ((138.38006 34.97625, 138.37999 34.976...",2.0,静岡市葵区,追手町,0.0,NaN,NaN,180,68,112,111,180,2,5,3,3,3,3,7,13,10,4,12,9,18,27,22,10,82,88,39,68,-,1,3,2,-,1,2,9,5,2,3,1,8,13,6,4,33,31,12,112,2,4,-,1,3,2,5,4,5,2,9,8,10,14,16,6,49,57,27,107,64,29,10,3,1,169,1.5794392523,107,41,39,19,11,2,3,9,65,105,82,19,103,11,-,90,-,4,-,86,2,85,-,-,-,-,-,1,1,5,-,13,3,6,12,8,7,6,11,-,7,2,3,62,17,4,85,1,19,18,15,22,1,-,2,-,-,4,3,107,-,-,61,44,2
1,221010020,62198.313,89,"POLYGON ((138.38196 34.98145, 138.38143 34.981...",2.0,静岡市葵区,城内町,0.0,NaN,NaN,89,39,50,40,89,3,5,3,2,2,2,3,5,1,6,8,5,9,8,6,11,43,35,21,39,2,3,-,1,2,1,2,3,1,2,4,2,4,5,3,5,22,12,4,50,1,2,3,1,-,1,1,2,-,4,4,3,5,3,3,6,21,23,17,40,12,17,3,7,-,89,2.2250000000,40,28,25,12,8,3,3,7,28,40,38,2,40,40,-,-,-,-,-,-,-,41,-,-,-,-,4,3,-,1,-,9,6,6,-,2,2,3,3,-,1,1,-,36,5,-,41,1,7,9,12,3,-,-,2,1,3,3,-,40,-,-,24,16,-
2,221010030,143355.593,142,"POLYGON ((138.38727 34.97837, 138.38724 34.978...",2.0,静岡市葵区,駿府町,0.0,NaN,NaN,142,54,88,67,142,6,5,6,4,3,1,3,8,14,9,10,11,8,14,12,17,71,51,25,54,1,3,1,1,2,1,1,5,3,3,4,4,4,5,7,5,28,20,8,88,5,2,5,3,1,-,2,3,11,6,6,7,4,9,5,12,43,31,17,67,23,25,9,8,2,142,2.1194029851,67,43,34,14,10,9,4,12,36,66,51,14,66,45,1,19,-,19,-,-,1,79,-,-,-,-,3,2,-,1,1,29,1,2,6,14,5,3,2,-,3,1,6,38,25,12,79,-,6,8,26,21,-,-,6,-,2,4,6,67,-,-,40,23,4
3,221010040,190501.969,0,"POLYGON ((138.38404 34.98125, 138.38562 34.980...",2.0,静岡市葵区,駿府公園,0.0,NaN,NaN,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
4,221010050,165730.625,1209,"POLYGON ((138.37944 34.97951, 138.37932 34.979...",2.0,静岡市葵区,西草深町,0.0,NaN,NaN,1209,573,636,523,1209,34,48,41,51,36,43,56,64,97,78,74,93,91,107,101,123,683,394,186,573,17,27,24,26,19,23,28,30,43,40,35,46,40,52,48,68,330,168,68,636,17,21,17,25,17,20,28,34,54,38,39,47,51,55,53,55,353,226,118,523,166,164,98,65,21,1209,2.3116634799,523,352,302,127,130,50,34,95,271,517,332,150,508,343,2,162,14,133,15,-,1,593,4,2,-,-,29,59,4,24,13,109,21,30,39,37,21,25,77,4,35,27,35,443,84,37,593,39,142,126,96,59,4,4,40,8,18,22,35,523,1,2,343,155,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

# できたデータを保存

In [ ]:
data2.to_file('/content/drive/MyDrive/work/pycon_mini_shizu/data/data/shizuoka_merged_data.geojson', driver='GeoJSON')

In [ ]:
data = gpd.read_file('/content/drive/MyDrive/work/pycon_mini_shizu/data/data/shizuoka_merged_data.geojson')

In [ ]:
data

,KEY_CODE,AREA,JINKO,HYOSYO,CITYNAME,NAME,HTKSYORI,HTKSAKI,GASSAN,人口総数,男,女,世帯総数,総数、年齢「不詳」含む,総数０～４歳,総数５～９歳,総数１０～１４歳,総数１５～１９歳,総数２０～２４歳,総数２５～２９歳,総数３０～３４歳,総数３５～３９歳,総数４０～４４歳,総数４５～４９歳,総数５０～５４歳,総数５５～５９歳,総数６０～６４歳,総数６５～６９歳,総数７０～７４歳,総数１５歳未満,総数１５～６４歳,総数６５歳以上,総数７５歳以上,男の総数、年齢「不詳」含む,男０～４歳,男５～９歳,男１０～１４歳,男１５～１９歳,男２０～２４歳,男２５～２９歳,男３０～３４歳,男３５～３９歳,男４０～４４歳,男４５～４９歳,男５０～５４歳,男５５～５９歳,男６０～６４歳,男６５～６９歳,男７０～７４歳,男１５歳未満,男１５～６４歳,男６５歳以上,男７５歳以上,女の総数、年齢「不詳」含む,女０～４歳,女５～９歳,女１０～１４歳,女１５～１９歳,女２０～２４歳,女２５～２９歳,女３０～３４歳,女３５～３９歳,女４０～４４歳,女４５～４９歳,女５０～５４歳,女５５～５９歳,女６０～６４歳,女６５～６９歳,女７０～７４歳,女１５歳未満,女１５～６４歳,女６５歳以上,女７５歳以上,一般世帯数（世帯人員６人以上含む）,世帯人員１人,世帯人員２人,世帯人員３人,世帯人員４人,世帯人員５人,一般世帯人員,１世帯当り人員,一般世帯総数,親族世帯,核家族世帯,うち夫婦のみの世帯,うち夫婦と子供から成る世帯,核家族以外の世帯,６歳未満世帯員のいる一般世帯総数,１８歳未満世帯員のいる一般世帯総数,６５歳以上世帯員のいる一般世帯総数,住宅に住む一般世帯,持ち家,民営借家,主世帯数,一戸建,長屋建,共同住宅,共同住宅１・２階建,共同住宅３～５階建,共同住宅６～１０階建,共同住宅１１階建以上,その他,総数_x,Ａ 農業、林業,うち農業,Ｂ 漁業,Ｃ 鉱業、採石業、砂利採取業,Ｄ 建設業,Ｅ 製造業,Ｆ 電気・ガス・熱供給・水道業,Ｇ 情報通信業,Ｈ 運輸業、郵便業,Ｉ 卸売業、小売業,Ｊ 金融業、保険業,Ｋ 不動産業、物品賃貸業,Ｌ 学術研究、専門・技術サービス業,Ｍ 宿泊業、飲食サービス業,Ｎ 生活関連サービス業、娯楽業,Ｏ 教育、学習支援業,Ｐ 医療、福祉,Ｑ 複合サービス事業,Ｒ サービス業（他に分類されないもの）,Ｓ 公務（他に分類されるものを除く）,Ｔ 分類不能の産業,雇用者（役員を含む）,自営業主（家庭内職者を含む）,家族従業者,総数_y,Ａ 管理的職業従事者,Ｂ 専門的・技術的職業従事者,Ｃ 事務従事者,Ｄ 販売従事者,Ｅ サービス職業従事者,Ｆ 保安職業従事者,Ｇ 農林漁業従事者,Ｈ 生産工程従事者,Ｉ 輸送・機械運転従事者,Ｊ 建設・採掘従事者,Ｋ 運搬・清掃・包装等従事者,Ｌ 分類不能の職業,総数,農林漁業就業者世帯,農林漁業・非農林漁業就業者混合世帯,非農林漁業就業者世帯,非就業者世帯,分類不能の世帯,geometry
0,221010010,175736.710,180,2.0,静岡市葵区,追手町,0.0,None,None,180,68,112,111,180,2,5,3,3,3,3,7,13,10,4,12,9,18,27,22,10,82,88,39,68,-,1,3,2,-,1,2,9,5,2,3,1,8,13,6,4,33,31,12,112,2,4,-,1,3,2,5,4,5,2,9,8,10,14,16,6,49,57,27,107,64,29,10,3,1,169,1.5794392523,107,41,39,19,11,2,3,9,65,105,82,19,103,11,-,90,-,4,-,86,2,85,-,-,-,-,-,1,1,5,-,13,3,6,12,8,7,6,11,-,7,2,3,62,17,4,85,1,19,18,15,22,1,-,2,-,-,4,3,107,-,-,61,44,2,"POLYGON ((138.38006 34.97625, 138.37999 34.976..."
1,221010020,62198.313,89,2.0,静岡市葵区,城内町,0.0,None,None,89,39,50,40,89,3,5,3,2,2,2,3,5,1,6,8,5,9,8,6,11,43,35,21,39,2,3,-,1,2,1,2,3,1,2,4,2,4,5,3,5,22,12,4,50,1,2,3,1,-,1,1,2,-,4,4,3,5,3,3,6,21,23,17,40,12,17,3,7,-,89,2.2250000000,40,28,25,12,8,3,3,7,28,40,38,2,40,40,-,-,-,-,-,-,-,41,-,-,-,-,4,3,-,1,-,9,6,6,-,2,2,3,3,-,1,1,-,36,5,-,41,1,7,9,12,3,-,-,2,1,3,3,-,40,-,-,24,16,-,"POLYGON ((138.38196 34.98145, 138.38143 34.981..."
2,221010030,143355.593,142,2.0,静岡市葵区,駿府町,0.0,None,None,142,54,88,67,142,6,5,6,4,3,1,3,8,14,9,10,11,8,14,12,17,71,51,25,54,1,3,1,1,2,1,1,5,3,3,4,4,4,5,7,5,28,20,8,88,5,2,5,3,1,-,2,3,11,6,6,7,4,9,5,12,43,31,17,67,23,25,9,8,2,142,2.1194029851,67,43,34,14,10,9,4,12,36,66,51,14,66,45,1,19,-,19,-,-,1,79,-,-,-,-,3,2,-,1,1,29,1,2,6,14,5,3,2,-,3,1,6,38,25,12,79,-,6,8,26,21,-,-,6,-,2,4,6,67,-,-,40,23,4,"POLYGON ((138.38727 34.97837, 138.38724 34.978..."
3,221010040,190501.969,0,2.0,静岡市葵区,駿府公園,0.0,None,None,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,"POLYGON ((138.38404 34.98125, 138.38562 34.980..."
4,221010050,165730.625,1209,2.0,静岡市葵区,西草深町,0.0,None,None,1209,573,636,523,1209,34,48,41,51,36,43,56,64,97,78,74,93,91,107,101,123,683,394,186,573,17,27,24,26,19,23,28,30,43,40,35,46,40,52,48,68,330,168,68,636,17,21,17,25,17,20,28,34,54,38,39,47,51,55,53,55,353,226,118,523,166,164,98,65,21,1209,2.3116634799,523,352,302,127,130,50,34,95,271,517,332,150,508,343,2,162,14,133,15,-,1,593,4,2,-,-,29,59,4,24,13,109,21,30,39,37,21,25,77,4,35,27,35,443,84,37,593,39,142,126,96,59,4,4,40,8,18,22,35,523,1,2,343,155,22,"POLYGON ((138.37944 34.97951, 138.37932 34.979..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
data['KEY_CODE'].value_counts()

22213075002    1
22211214002    1
221010610      1
22203027224    1
221350580      1
              ..
222250330      1
22226034009    1
22213150046    1
22216011103    1
221020160      1
Name: KEY_CODE, Length: 4777, dtype: int64

In [ ]:
data.iloc[:, 9: -1] = data.iloc[:, 9: -1].replace('-', '0').replace('X', '0')
data.iloc[:, 9: -1] = data.iloc[:, 9: -1].astype(float)

In [ ]:
data.to_file('/content/drive/MyDrive/work/pycon_mini_shizu/data/data/shizuoka_merged_data.geojson', driver='GeoJSON')


In [ ]:
data.crs

<Geographic 2D CRS: EPSG:4612>
Name: JGD2000
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: Japan - onshore and offshore.
- bounds: (122.38, 17.09, 157.65, 46.05)
Datum: Japanese Geodetic Datum 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [ ]:
data1 = data.query('HYOSYO == 2').copy()
data1 = data1[['KEY_CODE', 'JINKO', 'geometry']]
data1 = data1.set_crs('EPSG:4612', allow_override=True)
data1 = data1.to_crs('EPSG:4326')

center = shapely.geometry.MultiPolygon(data1.geometry.values).centroid
m = folium.Map([center.y, center.x], 
                zoom_start=14,
                #tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
                #attr='google map'
               )

folium.Choropleth(
geo_data=data1,
data=data1,
columns=['KEY_CODE', 'JINKO'],
key_on=f'feature.properties.KEY_CODE',
fill_color='OrRd',
name='labels'
).add_to(m)

folium.GeoJson(
    data1, 
    tooltip=folium.features.GeoJsonTooltip(fields=['KEY_CODE', "JINKO"])
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
data.shape

(4777, 147)